In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import requests
from talib import abstract
import math

plt.style.use('fivethirtyeight')

In [2]:
dataframe = pd.read_csv('BNBUSDT.csv')
dataframe = dataframe.iloc[-20880:-1]
dataframe = dataframe.reset_index()
dataframe

index            open time   open   high    low  close
0      415877  2021-10-17 12:00:00  469.0  469.5  468.4  469.1
1      415878  2021-10-17 12:01:00  469.1  469.1  468.6  469.1
2      415879  2021-10-17 12:02:00  469.1  469.3  468.7  468.9
3      415880  2021-10-17 12:03:00  469.0  469.1  468.5  468.9
4      415881  2021-10-17 12:04:00  468.9  468.9  468.5  468.5
...       ...                  ...    ...    ...    ...    ...
20874  436751  2021-10-31 23:54:00  524.5  524.7  524.3  524.7
20875  436752  2021-10-31 23:55:00  524.6  525.2  524.6  525.2
20876  436753  2021-10-31 23:56:00  525.2  525.2  524.9  524.9
20877  436754  2021-10-31 23:57:00  524.9  524.9  524.1  524.4
20878  436755  2021-10-31 23:58:00  524.3  524.4  524.0  524.1

[20879 rows x 6 columns]

In [3]:
dataframe['sma3'] = abstract.SMA(dataframe, timeperiod=3)
dataframe['sma20'] = abstract.SMA(dataframe, timeperiod=20)
dataframe['sma150'] = abstract.SMA(dataframe, timeperiod=150)

In [4]:
def buy_sell(data):
    money = 1000
    sigPriceBuy = []
    sigPriceSell = []
    
    earns = pd.DataFrame(data = {
        'buy time': [],
        'buy': [],
        'sell': [],
        'sell time': [],
        'earn': [],
    })
    
    flag = 'sold'
    
    for i in range(len(data)):
        if i <= 150:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue
            
        #SELL
        if flag == 'bought':
            rowIndex = earns[earns['buy'].notna() & earns['sell'].isna()].index.item()
            variation = (data['close'][i] / earns['buy'][rowIndex]) - 1
            
            if variation >= 0.001 or variation <= -0.0025:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(data['close'][i])
                flag = 'sold'
                earns.loc[rowIndex, 'sell'] = data['close'][i]
                earns.loc[rowIndex, 'earn'] = data['close'][i] - earns.loc[rowIndex, 'buy'] 
                earns.loc[rowIndex, 'sell time'] = data['open time'][i]
                continue
            
#             is_up = data['sma9'][i - 1] >= data['sma20'][i -  1]
#             is_down = data['sma9'][i] < data['sma20'][i]

#             if is_down and is_up:
#                 sigPriceBuy.append(np.nan)
#                 sigPriceSell.append(data['close'][i])
#                 flag = 'sold'
#                 earns.loc[rowIndex, 'sell'] = data['close'][i]
#                 earns.loc[rowIndex, 'earn'] = data['close'][i] - earns.loc[rowIndex, 'buy'] 
#                 continue

            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue
                
        
        
        
        
        # BUY                 
        initial_value = data.iloc[i-150]
        current_value = data.iloc[i]
        
        is_price_going_down = initial_value['close'] >= current_value['close']

        if is_price_going_down:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue
            
        tangent = (data['sma150'][i] - data['sma150'][i-150]) / 150 
        tendency_degrees = math.degrees(math.atan(tangent))
        is_bull_tendency = tendency_degrees > 1
        
        if not is_bull_tendency:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue
        
        
        
        
        
        
        
        
#         if len(earns[earns['sell time'].notna()]['sell time']) > 0:
#             open_Time = datetime.strptime(data['open time'][i], '%Y-%m-%d %H:%M:%S')
#             sell_time = datetime.strptime(earns.iloc[-1]['sell time'], '%Y-%m-%d %H:%M:%S')
#             minutes_diff = (open_Time - sell_time).total_seconds() / 60
#             if minutes_diff < 20:
#                 sigPriceBuy.append(np.nan)
#                 sigPriceSell.append(np.nan)
#                 continue
        
        
        
        
        
            
        has_previous_crossed_150 = False
        for j in range(60):
            is_20_down = data['sma20'][i - j -1 ] <= data['sma150'][i - j - 1] 
            is_20_up = data['sma20'][i - j] <= data['sma150'][i - j] 
            is_3_down = data['sma3'][i - j -1 ] <= data['sma150'][i - j - 1] 
            is_3_up = data['sma3'][i - j] <= data['sma150'][i - j] 
            if (is_20_down and is_20_up) or (is_3_down and is_3_up) :
                has_previous_crossed_150 = True
                
        if has_previous_crossed_150:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue
        
        has_previous_crossed_3 = False
        crossed_count = 0
        for j in range(25, 0, -1):
            is_down = data['sma3'][i - j - 1] <= data['sma20'][i - j - 1]
            is_up = data['sma3'][i - j] > data['sma20'][i - j]
            if is_down and is_up:
                has_previous_crossed_3 = True
                crossed_count += 1

#       Talvez aqui seja melhor != 1
        if crossed_count == 0 or not has_previous_crossed_3:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue
            
        last_40_prices = data.iloc[i - 41: i - 1]['close']
        higher_price = max(last_40_prices)
        
        if current_value['close'] > higher_price:
            if flag == 'sold':
                sigPriceBuy.append(data['close'][i])
                sigPriceSell.append(np.nan)
                
                flag = 'bought'
                earns = earns.append({ 
                    'buy': data['close'][i],
                    'buy time': data['open time'][i]
                }, ignore_index=True)
                continue
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
                continue
        
        
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
       
    
    return (sigPriceBuy, sigPriceSell, earns)

In [5]:
result = buy_sell(dataframe)
dataframe['buy'] = result[0]
dataframe['sell'] = result[1]
earns = result[2]

In [6]:
print('Lucro', earns['earn'].sum())
total = earns['earn'].count()
print('Quantidade', earns['earn'].count())
wins = earns[earns['earn'] > 0].count()
p = (wins/total)['earn'] * 100
print(f'Porcentagem {p}%')
print('-------')

Lucro 4.999999999999943
Quantidade 75
Porcentagem 64.0%
-------


In [7]:
# df = dataframe.iloc[0:10]
# print(df)
# plt.figure(figsize = (50, 12.5))
# plt.plot(df['open time'], df['close'], label = 'BNB/BTC', alpha=0.30)
# plt.plot(df['open time'], df['sma3'], label = 'SMA 30', alpha=0.30)
# plt.plot(df['open time'], df['sma9'], label = 'SMA 30', alpha=0.30)
# plt.plot(df['open time'], df['sma20'], label = 'SMA 30', alpha=0.30)
# plt.plot(df['open time'], df['sma150'], label = 'SMA 100', alpha=0.30)
# plt.scatter(df['open time'], df['buy'], label = 'Buy', marker = '^', color = 'green')
# plt.scatter(df['open time'], df['sell'], label = 'Sell', marker = 'v', color = 'red')
# plt.title('BNB/BTC Close price')
# plt.xlabel('Date')
# plt.ylabel('Close price')
# # plt.legend(loc='upper left')
# plt.show()

In [8]:
earns = earns[earns['buy'].notna() | earns['sell'].notna()]
earns.to_csv("result.csv", index=False)